In [1]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf

In [2]:
raw_mal = pd.read_csv('../input/ml-final-dataset/Malware_Data.csv')
raw_benign = pd.read_csv('../input/ml-final-dataset/PlayStore Application.csv')

In [3]:
l = raw_benign.isnull().sum()
rem_col = []
for i,col in enumerate(raw_benign.columns):
    if l[i]>1000:
        rem_col.append(col)
raw_benign = raw_benign.drop(rem_col,axis=1)
raw_mal = raw_mal.drop(rem_col,axis=1)

In [4]:
no_features=350
no_samples = 10000

In [5]:
fin_mal = raw_mal.iloc[:,2:no_features+2]
fin_benign = raw_benign.iloc[:20000,2:no_features+2]
mal_labels = np.ones((fin_mal.shape[0],1))
benign_labels = np.zeros((fin_benign.shape[0],1))

In [1]:
print(mal_labels.shape[0])
print(benign_labels.shape[0])


NameError: name 'mal_labels' is not defined

In [2]:
total_data = pd.concat([fin_mal,fin_benign],axis=0)
total_labels = pd.concat([pd.DataFrame(mal_labels),pd.DataFrame(benign_labels)],axis=0)

NameError: name 'pd' is not defined

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(total_data,total_labels,test_size=0.3)

NameError: name 'total_data' is not defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, random_state=0)
clf.fit(total_data,total_labels)
clf.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=10)
score = cross_val_score(RandomForestClassifier(n_estimators=100, random_state=0),total_data,total_labels)
print(score)

In [ ]:
fin_mal = np.array(fin_mal)
fin_benign = np.array(fin_benign)
fin_benign.shape, fin_mal.shape

In [ ]:
    noise_dim = 128
    dense_dim = 1024
    batch_size = 64
    optimizer = tf.keras.optimizers.Adam(0.001)

In [ ]:
def generator():
    
    inp = tf.keras.layers.Input(shape=(no_features))
    noise = tf.keras.layers.Input(shape=(noise_dim))
    full_inp = tf.keras.layers.Concatenate(axis=1)([inp,noise])
    #d_a1 = tf.keras.layers.Dense(2048)(full_inp)
    d1 = tf.keras.layers.Dense(1024)(full_inp)
    fin = tf.keras.layers.Dense(no_features,activation='sigmoid')(d1)
    
    out = tf.keras.layers.Maximum()([inp,fin])
    
    model = tf.keras.Model([inp,noise],out)
    return model
    

In [ ]:
def discriminator():
    
    inp = tf.keras.layers.Input(shape=(no_features))
    d2 = tf.keras.layers.Dense(1024)(inp)
    
    
    out = tf.keras.layers.Dense(1)(d2)
    
    model = tf.keras.Model(inp,out)
    return model

In [ ]:
def gan_total(gen,disc):
    
    inp = tf.keras.layers.Input(shape=(no_features))
    noise = tf.keras.layers.Input(shape=(noise_dim))
    full_inp = tf.keras.layers.Concatenate(axis=1)([inp,noise])
    
    gen_mal = gen([inp,noise])
    disc.trainable = False
    
    disc_pred = disc(gen_mal)
    
    final_gan = tf.keras.Model([inp,noise],disc_pred)
    
    return final_gan

In [ ]:
    gen = generator()
    disc = discriminator()
    disc.compile(optimizer=optimizer,loss='mse')
    gan = gan_total(gen,disc)
    gan.compile(optimizer=optimizer,loss='mse')

In [ ]:
    def train_step():
    
        idx = np.random.randint(0, fin_mal.shape[0], batch_size)
        mal_samples = fin_mal[idx]
        noise = np.random.uniform(0, 1, (batch_size, noise_dim))
    
        idx = np.random.randint(0, fin_benign.shape[0], batch_size)
        ben_samples = fin_benign[idx]
        ben_labels = np.zeros((len(ben_samples),1))
    
        gen_samples = gen.predict([mal_samples, noise])
        mal_labels = clf.predict(np.ones(gen_samples.shape)*(gen_samples > 0.5))
    
        d_loss_real = disc.train_on_batch(gen_samples, mal_labels)
        d_loss_fake = disc.train_on_batch(ben_samples, ben_labels)
        d_loss = (np.mean(d_loss_real)+np.mean(d_loss_fake))/2
    
        idx = np.random.randint(0, fin_mal.shape[0], batch_size)
        mal_samples = fin_mal[idx]
        noise = np.random.uniform(0, 1, (batch_size, noise_dim))

        g_loss = gan.train_on_batch([mal_samples, noise], np.zeros((batch_size, 1)))
    
        noise = np.random.uniform(0, 1, (fin_mal.shape[0], noise_dim))
        gen_samples = gen.predict([fin_mal, noise])
        TPR = clf.score(np.ones(gen_samples.shape) * (gen_samples > 0.5), np.ones((len(fin_mal),1)))
    
        return d_loss,g_loss,TPR
    

In [ ]:
epochs = 20
tpr_plot = []
gen_loss_plot = []
disc_loss_plot= []
max_tpr = 1
for ep in range(epochs):
    
    no_steps = len(fin_mal)//batch_size
    
    for step in range(no_steps):
        d_loss,g_loss,TPR = train_step()
        print(". ",end='')
    
    if TPR <max_tpr:
        gen.save('generator.h5')
        max_tpr = TPR
    tpr_plot.append(TPR)
    gen_loss_plot.append(g_loss)
    disc_loss_plot.append(d_loss)
    
    print("\nEpoch:",ep," Disciminator Loss: {:.2f}".format(d_loss)," Generator Loss: {:.2f}".format(g_loss)," True Positive Rate: {:.2f}".format(TPR))

In [ ]:
## TPR plot
import matplotlib.pyplot as plt
plt.plot(range(0,len(tpr_plot)),tpr_plot)

In [ ]:
## Generator Plot
import matplotlib.pyplot as plt
plt.plot(range(0,len(tpr_plot)),gen_loss_plot)